#### Import Dependencies

In [91]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchaudio
from torch.nn.utils.rnn import pack_padded_sequence, pad_sequence

import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os

#### Global Constants

In [92]:
load_dotenv()
DOWNLOAD_FOLDER = os.getenv('DOWNLOAD_FOLDER')
SAMPLE_RATE = 22050

#### Util Functions / Classes

In [93]:
# Function to load an MP3 file and convert it to a mel spectrogram
def get_mel_spectrogram(filepath, sample_rate=22050, n_mels=128):
    waveform, sr = torchaudio.load(filepath)
    
    # Resample if necessary
    if sr != sample_rate:
        resampler = torchaudio.transforms.Resample(sr, sample_rate)
        waveform = resampler(waveform)
    
    # Convert to mel spectrogram
    mel_spectrogram = torchaudio.transforms.MelSpectrogram(
        sample_rate=sample_rate,
        n_mels=n_mels
    )(waveform)
    
    # Normalize spectrogram
    mel_spectrogram = (mel_spectrogram - mel_spectrogram.mean()) / (mel_spectrogram.std() + 1e-6)
    
    return mel_spectrogram


In [94]:
# Transforms audio signal to mel spectrogram
mel_spectrogram = torchaudio.transforms.MelSpectrogram(
    sample_rate=SAMPLE_RATE,
    n_fft=1024,
    hop_length=512,
    n_mels=128
)

In [95]:
# Dataset class to load data and process mel spectrograms
class SongDataset(Dataset):
    def __init__(self, csv_path):
        self.data = pd.read_csv(csv_path)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Song filename follows this format: (index)^(video id)^(title).mp3 --> 0^LlWGt_84jpg^Special Breed.mp3 (example)
        song_filename = str(idx) + '^' + self.data.iloc[idx]['videoID'] + '^' + self.data.iloc[idx]['title'] + '.mp3'
        song_filepath = DOWNLOAD_FOLDER + '/' + song_filename
        viewcount = self.data.iloc[idx]['views']

        # Get the mel spectrogram
        signal, sr = torchaudio.load(song_filepath)
        signal = self.resample(signal, sr)
        signal = self.mix_down(signal)
        signal = self.right_pad(signal)
        signal = mel_spectrogram(signal)
        
        return signal, torch.tensor(viewcount, dtype=torch.float32)
    
    def resample(self, signal, sr):
        if sr == SAMPLE_RATE:
            return signal
        resampler = torchaudio.transforms.Resample(sr, SAMPLE_RATE)
        return resampler(signal)
    
    def mix_down(self, signal):
        # Skip if signal is already Mono
        if signal.shape[0] == 1:
            return signal
        # Convert signal shape from (2, 16000) -> (1, 16000)
        return torch.mean(signal, dim=0, keepdim=True)
    
    def right_pad(self, signal):
        signal_length = signal.shape[1]
        if signal_length >= SAMPLE_RATE * 10 * 60: # Pad to 10 minutes
            return signal
        num_missing_samples = SAMPLE_RATE * 10 * 60 - signal_length
        last_dim_padding = (0, num_missing_samples)
        signal = torch.nn.functional.pad(signal, last_dim_padding)
        return signal

    # Slice the dataset using lower and upper bounds 
    def slice(self, lower: int, upper: int):
        self.data = self.data.iloc[lower:upper]

#### Model Training

In [96]:
class ViewCountPredictor(nn.Module):
    def __init__(self):
        super().__init__()
        
        # CNN for feature extraction
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d((2, 2))
        )
        
        # RNN to handle variable-length sequences
        self.rnn = nn.LSTM(input_size=128, hidden_size=64, num_layers=2, batch_first=True)
        
        # Fully connected layers
        self.fc = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, 1)
        )
    
    def forward(self, x):
        """
        x: [batch_size, 1, n_mels, time_steps]
        lengths: list of original lengths before padding (for packing sequences)
        """
        # Apply CNN to extract features
        batch_size, channels, n_mels, time_steps = x.size()
        x = x.view(batch_size, 1, n_mels, time_steps)  # Ensure single channel dimension for CNN
        x = self.cnn(x)  # Output shape: [batch_size, 128, new_n_mels, new_time_steps]
        
        # Global average pooling across frequency dimension (n_mels -> 1)
        x = x.mean(dim=2)  # Shape: [batch_size, 128, new_time_steps]

        # Permute to match RNN input format: [batch_size, time_steps, features]
        x = x.permute(0, 2, 1)  # Shape: [batch_size, new_time_steps, 128]
        
        # Pack the padded sequences for RNN processing
        lengths = [sequence.size(2) for sequence in x]
        packed_input = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)

        # Pass through the RNN
        packed_output, (hn, cn) = self.rnn(packed_input)
        
        # Use the last hidden state (from the last layer)
        x = hn[-1]  # Shape: [batch_size, hidden_size]
        
        # Pass through the fully connected layers for final prediction
        x = self.fc(x)  # Output shape: [batch_size, 1]
        
        return x


In [97]:
def train_one_epoch(model, data_loader, loss_fn, optimizer, device):
    running_loss = 0.0
    for mel_spectrogram, viewcount in data_loader:
        mel_spectrogram, viewcount = mel_spectrogram.to(device), viewcount.to(device)
        mel_spectrogram = mel_spectrogram.unsqueeze(1)
        viewcount = viewcount.unsqueeze(1)

        # Forward pass through the model
        output = model(mel_spectrogram)
        loss = loss_fn(output, viewcount)
            
        # Perform backprop and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

In [98]:
def train(model, train_loader, val_loader, loss_fn, optimizer, device, epochs=20, lr=1e-3):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        train_one_epoch(model, train_loader, loss_fn, optimizer, device)
        
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader)}")
        
        # Validation Step
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for mel_spectrogram, viewcount in val_loader:
                mel_spectrogram = mel_spectrogram.unsqueeze(1)
                viewcount = viewcount.unsqueeze(1)
                output = model(mel_spectrogram)
                loss = loss_fn(output, viewcount)
                val_loss += loss.item()
        
        print(f"Validation Loss: {val_loss / len(val_loader)}")

#### Training Hyperparameters

In [99]:
EPOCHS = 20
BATCH_SIZE = 32
LEARNING_RATE = 1e-3

#### Main Code

In [100]:
# Initializing Datasets and DataLoaders
dataset = SongDataset('data/songs_data.csv')
dataset.slice(0, 10)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Check if CUDA is available
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"Using {device} device")

# Initialize and train the model
model = ViewCountPredictor().to(device)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

train(model, train_loader, val_loader, loss_fn, optimizer, device, epochs=EPOCHS)

Using cuda device


ValueError: too many values to unpack (expected 4)